In [14]:
# this notebook is to utilised transfer learning to train a model on a dataset to classify medical image data on breast cancer
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import pandas as pd
import torchvision
from torchvision import models, transforms
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
import matplotlib.pyplot as plt
import os
from torchvision.io import read_image
import time
import copy
import multiprocessing as mp

In [15]:
# get root directory
root_dir = '/media/victoru/B612CEC512CE8A37/ai50/pytorch_test/data/breatcancer_data/archive'
id_dir = os.listdir(root_dir)
label_dir = os.listdir(os.path.join(root_dir,id_dir[0]))
sample_dir = os.listdir(os.path.join(root_dir,id_dir[0],label_dir[0]))
num_classes = len(label_dir)

# create transform for data augmentation: normalize, and convert to tensor
mean = np.array([0.5, 0.5, 0.5])
std = np.array([0.5, 0.5, 0.5])
data_transforms = transforms.Compose([
        transforms.Resize([50,50]),
        transforms.Normalize(mean, std)
    ])
# define hyperparameters
batch_size = 100
num_epochs = 3
random_seed = 42
shuffle_dataset = True
validation_split = 0.2

#show image
def imshow(inp, title):
    """Imshow for Tensor."""
    inp = inp.numpy().transpose((1, 2, 0))
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.imshow(inp)
    plt.title(title)
    plt.show()

In [16]:
#create dataset class for breast cancer data

class breastcancerDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        # get root directory
        self.root_dir = root_dir
        # get transform
        self.transform = transform
        # get list of label from directory under root_dir
        id_dir = os.listdir(root_dir)
        label_dir = os.listdir(os.path.join(root_dir,id_dir[0]))
        self.data = pd.DataFrame(columns=['id','label','image_name'])
        
        # loop through id in id_dir and label in label_dir to get image name and label
        for id in id_dir:
            for label in label_dir:
                tempdata = pd.DataFrame(columns=['id','label','image_name'])
                tempdata['image_name'] = os.listdir(os.path.join(root_dir, id, label))
                tempdata['id'] = id
                tempdata['label'] = label
                self.data = pd.concat([self.data, tempdata],ignore_index = True, axis = 0)
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # get image name and label from dataframe
        img_name = os.path.join(self.root_dir,self.data.iloc[idx, 0],self.data.iloc[idx, 1],self.data.iloc[idx, 2])
        image = read_image(img_name)
        label = torch.tensor(int(self.data.iloc[idx, 1]))
                
        if self.transform:
            image = self.transform(image)
            
        return image,label

In [17]:
# create dataset object
dataset = breastcancerDataset(root_dir,data_transforms)

# create data indices for training and validation splits
dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

# create samplers for training and validation splits
train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

print(dataset_size)

277524


In [18]:
#check lowest time to load data for given num_workers
lowesttime = 999999999
num_workers_lowest = 0

for num_workers in range (2, mp.cpu_count(),2):
    testloader = DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler, num_workers=num_workers)
    print('number of worker: {}'.format(num_workers))
    n_total_steps = len(testloader)
    start = time.time()
    for epoch in range(1,2):
        for i, data in enumerate(testloader,0):
            if (i+1) % 20 == 0:
                print ('Step [{}/{}]'.format(i+1, n_total_steps))
            if i/n_total_steps > 0.4:
                print("40 percent of data loaded")
                break
            else:
                pass
    end = time.time()
    percent = (i/n_total_steps)*100
    timetaken = end - start
    print("Finish with: {} second, num_workers={}".format(timetaken,num_workers))
    if timetaken < lowesttime:
        lowesttime = timetaken
        num_workers_lowest = num_workers

print("Lowest time taken: {} second, num_workers={}".format(lowesttime,num_workers_lowest))

number of worker: 2


TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/victoru/anaconda3/lib/python3.8/site-packages/torch/utils/data/_utils/worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/victoru/anaconda3/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 49, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/victoru/anaconda3/lib/python3.8/site-packages/torch/utils/data/_utils/fetch.py", line 49, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/tmp/ipykernel_148736/2020481451.py", line 33, in __getitem__
    image = self.transform(image)
  File "/home/victoru/anaconda3/lib/python3.8/site-packages/torchvision/transforms/transforms.py", line 61, in __call__
    img = t(img)
  File "/home/victoru/anaconda3/lib/python3.8/site-packages/torchvision/transforms/transforms.py", line 98, in __call__
    return F.to_tens

In [6]:
# create data loaders
train_loader = DataLoader(dataset, batch_size=batch_size, sampler=train_sampler, num_workers=num_workers_lowest)
valid_loader = DataLoader(dataset, batch_size=batch_size, sampler=valid_sampler)

In [5]:
#images, classes = next(iter(train_loader))
#print(images.shape[2])
#print(type(images))
#print(images.shape)
#imshow(torchvision.utils.make_grid(images), title=classes)

In [7]:
#setting up device to run on cuda
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [11]:
#create training loop function
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in range(2):
            if phase == 0:
                # set model to train mode for training phase
                model.train()
                dataloader = train_loader
            else:
                # set model to evaluate mode for validation phase
                model.eval()
                dataloader = valid_loader
            running_loss = 0.0
            running_corrects = 0

            # Iterate over databatchs for give phase
            n_total_steps = len(dataloader)
            for i, (inputs, labels) in enumerate(dataloader):
                inputs = inputs.to(device)
                labels = labels.to(device)
                # zero the parameter gradients
                optimizer.zero_grad()

                # enable gradient computation when phase is train and forward pass for both phase
                with torch.set_grad_enabled(phase == 0):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)
                    # backward pass only if phase is train
                    if phase == 0:
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                if (i+1) % 100 == 0:
                    print ('Step [{}/{}], Loss: {:.4f}'.format(i+1, n_total_steps, loss.item()))
            epoch_loss = running_loss / dataset_size[phase]
            epoch_acc = running_corrects.double() / dataset_size[phase]
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model and update best_acc if phase is validation and epoch_acc is greater than best_acc
            if phase == 1 and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
                print('Best Acc: {:.4f}'.format(best_acc))
            # adjust learning rate using scheduler if phase is validation
            if phase == 1:
                print('Scheduler step')
                scheduler.step()
        print()

    # record time to train model
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

In [12]:
# fine tune model
# define fine tuned model as pretrained resnet34
model_ft = models.resnet18(pretrained=True)

# freeze all existing parameters
for param in model_ft.parameters():
    param.requires_grad = False

#get number of input features of last layer
num_features = model_ft.fc.in_features

#replace last layer with new layer with number of classes as output
model_ft.fc = nn.Linear(num_features, num_classes)

# set model to current device
model_ft = model_ft.to(device)

# define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_ft.parameters(), lr=0.001)

# setup scheduler for learning rate decay
step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [13]:
model_ft = train_model(model_ft, criterion, optimizer, step_lr_scheduler, num_epochs=num_epochs)

Epoch 0/9
----------
Step [100/2221], Loss: 0.4503
Step [200/2221], Loss: 0.5397
Step [300/2221], Loss: 0.4428
Step [400/2221], Loss: 0.4109
Step [500/2221], Loss: 0.4071
Step [600/2221], Loss: 0.4730
Step [700/2221], Loss: 0.5564
Step [800/2221], Loss: 0.4827
Step [900/2221], Loss: 0.3717
Step [1000/2221], Loss: 0.3949
Step [1100/2221], Loss: 0.4035
Step [1200/2221], Loss: 0.3479
Step [1300/2221], Loss: 0.4144
Step [1400/2221], Loss: 0.4595
Step [1500/2221], Loss: 0.4589
Step [1600/2221], Loss: 0.4287
Step [1700/2221], Loss: 0.3485
Step [1800/2221], Loss: 0.3703
Step [1900/2221], Loss: 0.3214
Step [2000/2221], Loss: 0.4693
Step [2100/2221], Loss: 0.5643


KeyboardInterrupt: 